# 학습목표
scikeras GridSearch, keras tuner를 사용하여 하이퍼파라미터를 튜닝합니다 

In [84]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

import tensorflow as tf
import IPython

import numpy as np
import pandas as pd
import tensorflow as tf
import os

데이터 불러오기 

In [85]:
from tensorflow.keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

### 베이스라인 모델 

In [86]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [87]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mse'])

In [88]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
13/13 [==============================] - 1s 4ms/step - loss: 620.8333 - mse: 620.8333
Epoch 2/10
13/13 [==============================] - 0s 3ms/step - loss: 161.2493 - mse: 161.2493
Epoch 3/10
13/13 [==============================] - 0s 5ms/step - loss: 99.9419 - mse: 99.9419
Epoch 4/10
13/13 [==============================] - 0s 3ms/step - loss: 76.8446 - mse: 76.8446
Epoch 5/10
13/13 [==============================] - 0s 2ms/step - loss: 65.2172 - mse: 65.2172
Epoch 6/10
13/13 [==============================] - 0s 3ms/step - loss: 64.3626 - mse: 64.3626
Epoch 7/10
13/13 [==============================] - 0s 3ms/step - loss: 63.2421 - mse: 63.2421
Epoch 8/10
13/13 [==============================] - 0s 2ms/step - loss: 59.8862 - mse: 59.8862
Epoch 9/10
13/13 [==============================] - 0s 2ms/step - loss: 56.1960 - mse: 56.1960
Epoch 10/10
13/13 [==============================] - 0s 3ms/step - loss: 55.3543 - mse: 55.3543


In [89]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 59.3970 - mse: 59.3970


[59.39701461791992, 59.39701461791992]

### GridSearch 

In [90]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor #KerasClassifier

1. 신경망 모델 함수 형태로 정의 

In [92]:
numpy.random.seed(42)

In [136]:
# 신경망 모델을 함수 형태로 정의
def create_model(node1=8, node2=16):
  model = Sequential()
  model.add(Dense(node1, input_dim=13, activation='relu'))
  model.add(Dense(node2, activation='relu'))
  model.add(Dense(4))
  model.add(Dense(1, activation='sigmoid'))
    
  model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['mse'])
  return model

2. wrapping

In [137]:
# KerasRegressor wrapping
model = KerasRegressor(model=create_model, batch_size=8, verbose=False)


3. GridSearchCV 지정 

In [138]:
# 그리드서치 파라미터 지정 
param_grid = {'model__node1':[32, 64, 128],
              'model__node2':[32, 64, 128],
              'batch_size':[8, 32]}

In [139]:
# 그리드서칭
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3)

grid_result = grid.fit(X_train, y_train)

In [140]:
grid_result

GridSearchCV(cv=3,
             estimator=KerasRegressor(batch_size=8, model=<function create_model at 0x7f10710de1f0>, verbose=False),
             n_jobs=-1,
             param_grid={'batch_size': [8, 32], 'model__node1': [32, 64, 128],
                         'model__node2': [32, 64, 128]})

In [141]:
grid_result.cv_results_

{'mean_fit_time': array([1.56436475, 1.01384004, 1.0623308 , 1.0612758 , 1.054691  ,
        1.08145714, 1.06683858, 1.00643086, 1.91402992, 0.97491805,
        0.97423561, 1.00313616, 1.00371583, 0.96615028, 0.96323029,
        0.94526958, 1.41110921, 0.97738942]),
 'std_fit_time': array([0.36344821, 0.03141051, 0.01127835, 0.00429932, 0.03736699,
        0.01820718, 0.01938543, 0.01909571, 0.63998985, 0.01837159,
        0.03687095, 0.01909059, 0.01613216, 0.00911519, 0.025886  ,
        0.01777019, 0.34492997, 0.00359496]),
 'mean_score_time': array([0.22790647, 0.2144018 , 0.17390744, 0.19746757, 0.21671112,
        0.20925458, 0.19540191, 0.22580791, 0.21015898, 0.16125822,
        0.20654869, 0.17685668, 0.18050949, 0.16720128, 0.16812833,
        0.15944266, 0.1646568 , 0.16024295]),
 'std_score_time': array([0.01793753, 0.0120088 , 0.01652318, 0.01294572, 0.02999127,
        0.00503334, 0.01687228, 0.01402306, 0.01118692, 0.00256524,
        0.00834462, 0.01291483, 0.02174033, 

In [142]:
grid_result.best_score_

-5.6715509726833275

In [143]:
grid_result.best_params_

{'batch_size': 8, 'model__node1': 32, 'model__node2': 128}

### Keras Tuner 

In [106]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

import tensorflow as tf
import IPython

In [107]:
!pip install -U keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00


1. 신경망 모델 함수 형태로 정의 

In [126]:
def model_builder(hp):
  model = Sequential()

  hp_unit = hp.Choice('units', [32, 64, 128])
  model.add(Dense(units=hp_unit, input_dim=13, activation='relu'))
  model.add(Dense(units=hp_unit, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
    
  hp_lr = hp.Choice('learning_rate', values = [0.01, 0.05, 0.1])
  model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adam(learning_rate = hp_lr),
              metrics=['mse'])
  return model

2. tuner 정의

In [127]:
tuner = kt.Hyperband(model_builder,
                     objective = 'mse', 
                     max_epochs = 10,
                     factor = 3,
                     project_name = 'intro_to_kt')   

3. 콜백 지정
하이퍼파라미터 학습 실행 전, 이전 학습 출력을 제거하는 콜백 함수 

In [128]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

4. keras tuner 서칭 시작 

In [129]:
tuner.search(X_train, y_train,
             epochs = 10,
             validation_data = (X_test, y_test),
             callbacks = [ClearTrainingOutput()])

In [130]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [131]:
best_hps

In [132]:
best_hps.get('units')

32

In [133]:
best_hps.get('learning_rate')

0.05

5. 최고 성능 모델 확인 

In [134]:
model = tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 32)                448       
                                                                 
 dense_13 (Dense)            (None, 32)                1056      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________


In [135]:
model.fit(X_train, y_train,
          epochs=10)
model.evaluate(X_test, y_test)

Epoch 1/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 2/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 3/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 4/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 5/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 6/10
13/13 [==============================] - 0s 2ms/step - loss: 586.1605 - mse: 586.1605
Epoch 7/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 8/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 9/10
13/13 [==============================] - 0s 3ms/step - loss: 586.1605 - mse: 586.1605
Epoch 10/10
4/4 [==============================] - 0s 4ms/step - loss: 615.8578 - mse: 615.8578


[615.8577880859375, 615.8577880859375]